In [102]:
import pandas as pd

In [2]:
df = pd.read_csv('tweet_full_combined_no_dup.csv')

In [3]:
df_no_null = df[~(df.location.isnull())]

In [5]:
location_needed = list(set(df_no_null.author_id))

In [6]:
# assign the values accordingly
consumer_key = ""
consumer_secret = ""
access_token = ""
access_token_secret = ""

# authorization of consumer key and consumer secret
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)

# set access to user's access key and access secret 
auth.set_access_token(access_token, access_token_secret)

# calling the api 
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [8]:
dict_location = {}

In [52]:
for id1 in location_needed:
    try:
        user = api.get_user(id1)
        dict_location.update({id1: user.status.geo})
    except:
        try:
            dict_location.update({id1: user.location})
        except:
            continue

Rate limit reached. Sleeping for: 357


In [56]:
df_no_null['location'] = df_no_null['author_id'].map(dict_location)

C:\Users\chaewon7\AppData\Local\Temp\ipykernel_6684\426542110.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_null_spaces_only['location'] = df_no_null_spaces_only['author_id'].map(dict_location)


In [184]:
#resolve coordinates_point to exact address
from geopy.geocoders import GoogleV3

In [223]:
geolocator = GoogleV3(api_key='')

In [216]:
df_with_coor = df[~(df.coordinates_point.isnull())]
df_no_coor = df[(df.coordinates_point.isnull())]

In [217]:
df_with_coor_sep = pd.DataFrame(df_with_coor['coordinates_point'].to_list(), columns=['longitude','latitude'])

In [1]:
df_with_coor['longitude'] = df_with_coor_sep['longitude'].to_list()
df_with_coor['latitude'] = df_with_coor_sep['latitude'].to_list()

In [2]:
#latitude, longitude
df_with_coor.longitude = df_with_coor.longitude.astype(str)
df_with_coor.latitude = df_with_coor.latitude.astype(str)

In [3]:
df_with_coor['lat_long'] = df_with_coor['latitude'] + ', ' + df_with_coor['longitude']

In [246]:
df_with_coor = df_with_coor.reset_index(drop=True)

In [347]:
df_with_coor['address'] = df_with_coor['author_id'].map(reverse_geocoding)

In [423]:
df_combined = pd.concat([df_with_coor, df_no_coor])

In [424]:
#exclude GDPR using coordinates

In [426]:
df_combined.location = df_combined.location.str.lower()
df_combined.address = df_combined.address.str.lower()

In [429]:
eu = ['austria', 'belgium', 'bulgaria', 'croatia', 'cyprus', 'republic of cyprus', 'czech', 'czech republic', 'denmark', 
           'estonia', 'finland', 'france', 'germany', 'greece', 'hungary', 'ireland', 'italy', 'latvia', 'lithuania', 
           'luxembourg', 'malta', 'netherlands', 'poland', 'portugal', 'romania', 'slovakia', 'slovenia', 'spain', 'sweden', 
           'united kingdom', 'england', 'great britain']

In [430]:
#no gdpr data
df_combined_no_eu = df_combined[~(df_combined.address.str.contains('|'.join(eu), na=False)) & ~(df_combined_no_eu.location.str.contains('|'.join(eu), na=False))]

In [ ]:
df_combined_no_eu.to_csv('no_gdpr.csv', index=False)